<a href="https://colab.research.google.com/github/injetiharsha/Crop-Disease-Prediction-/blob/main/Crop_Detection_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Step 1: Restart Runtime
#import os
#os._exit(0)  # This will force restart the runtime


In [3]:
# Step 2: Reinstall the correct versions
!pip install --upgrade sympy torchvision torch

# Step 3: Import everything again (after installation)
import torch
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F

# Step 4: Check if everything works
print("✅ Torch and torchvision loaded successfully!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [7]:
import os

dataset_zip_path = "/Plant_leave_diseases_dataset_without_augmentation.zip"

# Check file type
if os.path.exists(dataset_zip_path):
    print("File exists! Checking type...")
    !file /content/dataset.zip
else:
    print("File does not exist!")


File exists! Checking type...
/content/dataset.zip: empty


In [8]:
import os
import zipfile
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Define dataset zip file path
dataset_zip_path = "/Plant_leave_diseases_dataset_without_augmentation.zip"

# Extract dataset if not already extracted
dataset_root = "/content/Plant_leave_diseases_dataset_without_augmentation"  # Check actual folder name

if not os.path.exists(dataset_root):
    print("Extracting dataset...")
    with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content/")
else:
    print("Dataset already extracted. Skipping extraction.")

# Verify extracted files
print("Extracted files:", os.listdir("/content/"))

# Define the correct dataset path
dataset_path = dataset_root  # Adjust if images are inside a subfolder

if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset folder not found at {dataset_path}. Check extracted files.")

# Define Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor()           # Convert to tensor
])

# Load Dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Verify Dataset Loaded
print("✅ Dataset loaded successfully!")
print(f"Total Images: {len(dataset)}")
print(f"Classes: {dataset.classes}")


Extracting dataset...
Extracted files: ['.config', 'dataset.zip', 'Plant_leave_diseases_dataset_without_augmentation', 'sample_data']
✅ Dataset loaded successfully!
Total Images: 16223
Classes: ['Background_without_leaves', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


### Import Dataset

In [9]:
indices = list(range(len(dataset)))

In [10]:
import numpy as np

split = int(np.floor(0.85 * len(dataset)))  # train_size

In [11]:
validation = int(np.floor(0.70 * split))  # validation

In [12]:
print(0, validation, split, len(dataset))

0 9652 13789 16223


In [13]:
print(f"length of train size :{validation}")
print(f"length of validation size :{split - validation}")
print(f"length of test size :{len(dataset)-validation}")

length of train size :9652
length of validation size :4137
length of test size :6571


In [14]:
np.random.shuffle(indices)

### Split into Train and Test

In [15]:
train_indices, validation_indices, test_indices = (
    indices[:validation],
    indices[validation:split],
    indices[split:],
)

In [16]:
train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(validation_indices)
test_sampler = SubsetRandomSampler(test_indices)

In [17]:
targets_size = len(dataset.class_to_idx)

### Model

<b>Convolution Aithmetic Equation : </b>(W - F + 2P) / S + 1 <br>
W = Input Size<br>
F = Filter Size<br>
P = Padding Size<br>
S = Stride <br>

### Transfer Learning

In [ ]:
# model = models.vgg16(pretrained=True)

In [ ]:
# for params in model.parameters():
#     params.requires_grad = False

In [ ]:
# model

In [ ]:
# n_features = model.classifier[0].in_features
# n_features

In [ ]:
# model.classifier = nn.Sequential(
#     nn.Linear(n_features, 1024),
#     nn.ReLU(),
#     nn.Dropout(0.4),
#     nn.Linear(1024, targets_size),
# )

In [ ]:
# model

### Original Modeling

In [18]:
class CNN(nn.Module):
    def __init__(self, K):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            # conv1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            # conv2
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            # conv3
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            # conv4
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
        )

        self.dense_layers = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(50176, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 39),
        )

    def forward(self, X):
        out = self.conv_layers(X)

        # Flatten
        out = out.view(-1, 50176)

        # Fully connected
        out = self.dense_layers(out)

        return out

In [19]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [20]:
device = "cuda"

In [21]:
model = CNN(targets_size)

In [22]:
model.to(device)

CNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

In [23]:
!pip install torchsummary


from torchsummary import summary

summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
              ReLU-2         [-1, 32, 224, 224]               0
       BatchNorm2d-3         [-1, 32, 224, 224]              64
            Conv2d-4         [-1, 32, 224, 224]           9,248
              ReLU-5         [-1, 32, 224, 224]               0
       BatchNorm2d-6         [-1, 32, 224, 224]              64
         MaxPool2d-7         [-1, 32, 112, 112]               0
            Conv2d-8         [-1, 64, 112, 112]          18,496
              ReLU-9         [-1, 64, 112, 112]               0
      BatchNorm2d-10         [-1, 64, 112, 112]             128
           Conv2d-11         [-1, 64, 112, 112]          36,928
             ReLU-12         [-1, 64, 112, 112]               0
      BatchNorm2d-13         [-1, 64, 112, 112]             128
        MaxPool2d-14           [-1, 64,

In [24]:
criterion = nn.CrossEntropyLoss()  # this include softmax + cross entropy loss
optimizer = torch.optim.Adam(model.parameters())

In [25]:
device = "cuda"

In [26]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=train_sampler,pin_memory=True
)
test_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=test_sampler
)
validation_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=validation_sampler
)

### Batch Gradient Descent

In [27]:
from tqdm import tqdm
from sklearn.metrics import f1_score

def calculate_f1(model, data_loader, device):
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)  # Get predicted class

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return f1_score(all_labels, all_preds, average="weighted")  # 'weighted' handles class imbalance



In [28]:
import datetime


def batch_gd(model, criterion, train_loader, validation_loader, epochs):
    train_losses = []  # Change from np.zeros() to a list
    validation_losses = []
    f1_scores = []  # Store F1 scores

    for e in range(epochs):
        t0 = datetime.now()
        train_loss = []

        # Training loop with progress bar
        model.train()
        for inputs, targets in tqdm(train_loader, desc=f"Epoch {e+1}/{epochs}"):
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, targets)
            train_loss.append(loss.item())

            loss.backward()
            optimizer.step()

        train_loss = np.mean(train_loss)
        train_losses.append(train_loss)  # ✅ Store train loss

        # Validation loop
        model.eval()
        validation_loss = []
        for inputs, targets in validation_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            output = model(inputs)
            loss = criterion(output, targets)
            validation_loss.append(loss.item())

        validation_loss = np.mean(validation_loss)
        validation_losses.append(validation_loss)  # ✅ Store validation loss

        # Calculate and store F1-score after each epoch
        f1 = calculate_f1(model, validation_loader, device)
        f1_scores.append(f1)

        print(f"Epoch {e+1}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {validation_loss:.4f} | F1-Score: {f1:.4f} | Duration: {datetime.now() - t0}")

    return train_losses, validation_losses, f1_scores


In [29]:
from datetime import datetime  # ✅ Import the class, not the module

current_time = datetime.now()
print("Current Time:", current_time)

print(torch.cuda.memory_summary(device=None, abbreviated=False))

train_losses, validation_losses, f1_scores = batch_gd(model, criterion, train_loader, validation_loader, 10)


Current Time: 2025-04-03 17:54:12.356472
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 213789 KiB | 329259 KiB | 390842 KiB | 177053 KiB |
|       from large pool | 212480 KiB | 324448 KiB | 383202 KiB | 170722 KiB |
|       from small pool |   1309 KiB |   4811 KiB |   7640 KiB |   6331 KiB |
|---------------------------------------------------------------------------|
| Active memory         | 213789 KiB | 329259 KiB | 390842 KiB | 177053 KiB |
|       from large pool

Epoch 1/10: 100%|██████████| 302/302 [00:55<00:00,  5.43it/s]


Epoch 1/10 | Train Loss: 2.2419 | Val Loss: 0.9233 | F1-Score: 0.7655 | Duration: 0:01:26.823616


Epoch 2/10: 100%|██████████| 302/302 [00:55<00:00,  5.46it/s]


Epoch 2/10 | Train Loss: 0.8653 | Val Loss: 0.7195 | F1-Score: 0.7739 | Duration: 0:01:26.587970


...

Epoch 8/10 | Train Loss: 0.8553 | Val Loss: 1.6482 | F1-Score: 0.6854 | Duration: 0:01:28.594678


Epoch 9/10: 100%|██████████| 302/302 [00:56<00:00,  5.30it/s]


Epoch 9/10 | Train Loss: 0.6875 | Val Loss: 0.3855 | F1-Score: 0.8744 | Duration: 0:01:28.567984


Epoch 10/10: 100%|██████████| 302/302 [00:56<00:00,  5.30it/s]


Epoch 10/10 | Train Loss: 0.4726 | Val Loss: 0.4121 | F1-Score: 0.8830 | Duration: 0:01:29.301214


### Save the Model

In [30]:
torch.save(model.state_dict() , 'plant_disease_model_14.pth')

### Load Model

In [31]:
from google.colab import files

files.download("plant_disease_model_14.pth")  # Download the model to your local machine


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
targets_size = 9
model = CNN(targets_size)
model.load_state_dict(torch.load("plant_disease_model_14.pth"))  # Load weights
model.to(device)  # Move to GPU if needed


CNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

In [43]:
%matplotlib notebook

### Accuracy

In [35]:
def accuracy(loader):
    model.eval()  # Switch to evaluation mode
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            pred = outputs.argmax(dim=1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)
    model.train()  # Switch back to training mode
    return correct / total


In [36]:
train_accuracies = []
validation_accuracies = []
test_accuracies = []

epochs = 10  # Define the number of epochs

for e in range(epochs):
    train_accuracies.append(accuracy(train_loader))
    validation_accuracies.append(accuracy(validation_loader))
    test_accuracies.append(accuracy(test_loader))

print("Training Accuracies:", train_accuracies)
print("Validation Accuracies:", validation_accuracies)
print("Test Accuracies:", test_accuracies)


KeyboardInterrupt: 

In [37]:
print(
    f"Train Accuracy : {train_accuracies}\nTest Accuracy : {test_accuracies}\nValidation Accuracy : {validation_accuracies}"
)

Train Accuracy : [0.9067550766680481, 0.9067550766680481, 0.9067550766680481, 0.9067550766680481, 0.9067550766680481, 0.9067550766680481]
Test Accuracy : [0.8915365653245686, 0.8915365653245686, 0.8915365653245686, 0.8915365653245686, 0.8915365653245686]
Validation Accuracy : [0.8834904520183708, 0.8834904520183708, 0.8834904520183708, 0.8834904520183708, 0.8834904520183708]


### Plot the loss

In [46]:
# Compute accuracy on trained model (No Retraining)
print("Computing accuracies without retraining...\n")

# Store accuracies
train_acc_list = []
val_acc_list = []
test_acc_list = []

# Calculate accuracies
train_acc = accuracy(eval_train_loader)
val_acc = accuracy(eval_validation_loader)
test_acc = accuracy(eval_test_loader)

# Append to lists
train_acc_list.append(train_acc)
val_acc_list.append(val_acc)
test_acc_list.append(test_acc)

# Print computed accuracy
print(f"Training Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# Plot Accuracy Graph
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_acc_list) + 1), train_acc_list, label='Train Accuracy', marker='o', linestyle='dashed')
plt.plot(range(1, len(val_acc_list) + 1), val_acc_list, label='Validation Accuracy', marker='s', linestyle='solid')
plt.plot(range(1, len(test_acc_list) + 1), test_acc_list, label='Test Accuracy', marker='^', linestyle='solid')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy vs. Epochs (After Training)")
plt.legend()
plt.grid(True)
plt.show()


Computing accuracies without retraining...

Training Accuracy: 0.9086
Validation Accuracy: 0.8874
Test Accuracy: 0.8989


<IPython.core.display.Javascript object>

In [47]:
# Save the evaluated model
torch.save(model.state_dict(), "evaluated_model.pth")

# Verify if the file exists
import os
if os.path.exists("evaluated_model.pth"):
    print("✅ Model saved successfully!")
else:
    print("❌ Model save failed!")


✅ Model saved successfully!


In [48]:
from google.colab import files
files.download("evaluated_model.pth")  # Change filename if using full model


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [49]:
from google.colab import drive
drive.mount('/content/drive')

# Save model to Drive
torch.save(model.state_dict(), "/content/drive/My Drive/evaluated_model.pth")


Mounted at /content/drive


In [50]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import requests
from io import BytesIO

# Step 1: Load the trained model
model = CNN(targets_size)  # Ensure this matches your defined model class
model.load_state_dict(torch.load("evaluated_model.pth", map_location=device))  # Load trained weights
model.to(device)
model.eval()  # Set model to evaluation mode

# Step 2: Define image preprocessing (same as used in training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match model input
    transforms.ToTensor(),  # Convert to tensor
])

# Step 3: Download the image from URL
image_url = "https://github.com/manthan89-py/Plant-Disease-Detection/blob/main/test_images/Apple_ceder_apple_rust.JPG?raw=true"  # Replace with actual image URL
response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert("RGB")  # Open and convert to RGB

# Step 4: Apply transformations and add batch dimension
image = transform(image).unsqueeze(0).to(device)  # Add batch dimension & move to GPU

# Step 5: Make a prediction
with torch.no_grad():
    output = model(image)  # Forward pass
    _, predicted_class = torch.max(output, 1)  # Get the predicted class index

# Step 6: Display the result
class_names = dataset.classes  # Get class names from dataset
predicted_label = class_names[predicted_class.item()]  # Convert index to label

print(f"Predicted Class: {predicted_label}")


Predicted Class: Tomato___Bacterial_spot


In [52]:
from google.colab import files
uploaded = files.upload()

data = pd.read_csv("disease_info.csv", encoding="cp1252")

Saving disease_info.csv to disease_info.csv


In [53]:
!pip install pillow torchvision

from PIL import Image
import torchvision.transforms.functional as TF